In [ ]:
import pandas as pd
import numpy as np

In [ ]:
lst = []
for i in ['Westlake', 'stouffville']:
    for j in ['business', 'checkin', 'review', 'tip', 'user']:
        f = open(f'yelp-all/{i}/{j}.json')
        lst.append(pd.read_json(f, lines=True))

Westlake_business = lst[0]
Westlake_checkin = lst[1]
Westlake_review = lst[2]
Westlake_tip = lst[3]
Westlake_user = lst[4]
Sun_business = lst[5]
Sun_checkin = lst[6]
Sun_review = lst[7].sort_values(['user_id'])
Sun_tip = lst[8]
Sun_user = lst[9]
Sun_review

In [ ]:
# define a helper function for accessing data
def get_rating(reviews, user_id = None, business_id = None, review_id = None):
    try:
        if user_id and business_id:
            return reviews[(reviews['user_id'] == user_id) & (reviews['business_id'] == business_id)]['stars'].values.mean()
        elif review_id:
            return reviews[(reviews['review_id'] == review_id)]
    except:
        return np.nan# define a helper function to create a pivot table

In [ ]:
def pivot_ratings(reviews):
    businessIds = reviews['business_id'].unique()
    userIds = reviews['user_id'].unique()
    
    # create empty data frame
    pivot_data = pd.DataFrame(np.nan, columns=userIds, index=businessIds, dtype=float)
    
    # use the function get_rating to fill the matrix
    for i in pivot_data.columns:
        for j in pivot_data.index:
            pivot_data[i][j] = get_rating(reviews, i, j)
    
    return pivot_data



%time utility_matrix = pivot_ratings(Sun_review).T #transpose is necessary for user-based
display(utility_matrix)


In [ ]:
def cosine_similarity(matrix, id1, id2):
    """Compute cosine similarity"""
    selected_features = matrix.loc[id1].notna() & matrix.loc[id2].notna()
    
    # if no matching features, return 0
    if not selected_features.any():
        return 0
    
    # get the features from the matrix
    features1 = matrix.loc[id1][selected_features]
    features2 = matrix.loc[id2][selected_features]
    
    # return 1 for the diagonals and 0 if there are no matching features
    if features1.equals(features2):
        return 1
    if features1.max() == 0 or features2.max() == 0:
        return 0
    
    return sum(features1 * features2)/((sum(features1**2)**0.5) * sum((features2**2))**0.5)


def create_similarity_matrix_cosine(matrix):
    """ creates the similarity matrix based on cosine similarity """
    similarity_matrix = pd.DataFrame(0, index=matrix.index, columns=matrix.index, dtype=float)
    id1 = similarity_matrix.columns.values
    for i in id1:
        for j in id1:
            similarity_matrix[i][j] = cosine_similarity(matrix, i, j)
    return similarity_matrix

def mean_center_columns(matrix):
    for column in matrix.columns:
        matrix[column] -= matrix[column].mean()
    return matrix

def select_neighborhood(similarity_matrix, utility_matrix, target_user, target_business):
    """selects all items with similarity > 0"""
    try:
        all_neighbors = similarity_matrix.loc[utility_matrix[target_user].dropna().index.values][target_business]
        return all_neighbors[all_neighbors > 0]
    except:
        return np.nan

def weighted_mean(neighborhood, utility_matrix, user_id):
    try:
        ratings = utility_matrix[user_id][neighborhood.index.values]
        return (neighborhood * ratings).sum() / neighborhood.sum()
    except:
        return np.nan
    
centered_utility_matrix = mean_center_columns(utility_matrix)
# display(centered_utility_matrix)

similarity_matrix = create_similarity_matrix_cosine(centered_utility_matrix)
display(similarity_matrix)